Reference: https://github.com/SuYenTing/Python-web-crawler

# 載入套件

In [1]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
import json
import datetime

# 參數設定

In [9]:
# 欲下載新聞的股票關鍵字清單
company_str = '鴻海	, 聯發科	, 台達電	, 聯電	, 中華電	, 南亞	, 富邦金	, 中信金	, 中鋼	, 兆豐金	, 玉山金	, 統一	, 台塑	, 國泰金	, 日月光投控	, 第一金	, 合庫金	, 台化	, 元大金	, 中租-KY	, 華南金	, 台泥	, 廣達	, 開發金	, 和泰車	, 大立光	, 永豐金	, 華碩	, 台灣大	, 統一超	, 上海商銀	, 台新金	, 欣興	, 聯詠	, 長榮	, 國巨	, 臺股期貨	, 友達	, 矽力*-KY	, 研華	, 瑞昱	, 亞德客-KY	, 彰銀	, 遠傳	, 陽明	, 台塑化	, 萬海	, 豐泰	, 南亞科	, 南電	,'
searchList = company_str.strip('\t,').split('\t, ')
searchList.append('台灣50')
# 新聞下載起始日
StartDate = '2010-01-01'

# 相關函數

In [3]:
# 整理Google新聞資料用
def arrangeGoogleNews(elem):
    return ([elem.find('title').getText(),
             elem.find('link').getText(),
             elem.find('pubDate').getText(),
             elem.find('description').getText(),
             elem.find('source').getText()])

In [4]:
# 擷取各家新聞網站新聞函數
# First part
def beautifulSoupNews(url):
    
    # 設定hearers
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.141 Safari/537.36'}

    # 下載網站原始碼
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        transfer_url = soup.find("meta", attrs={"property": "og:url"})['content']
        # 判斷url網域做對應文章擷取
        domain = re.findall('https://[^/]*', transfer_url)[0].replace('https://', '')
    except: 
        domain = "unknow"

    if domain == 'udn.com':

        # 聯合新聞網
        item = soup.find_all('section', class_='article-content__editor')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ')

    elif domain in ['tw.stock.yahoo.com', 'tw.news.yahoo.com']:

        # Yahoo奇摩
        item = soup.find_all('div', class_='caas-body')[0].find_all('p')
        content = [elem.getText() for elem in item]
        del_text = soup.find_all('div', class_='caas-body')[0].find_all('a')
        del_text = [elem.getText() for elem in del_text]
        content = [elem for elem in content if elem not in del_text]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')

    elif domain == 'money.udn.com':

        # 經濟日報
        item = soup.find_all('section', id='article_body')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = [elem for elem in content]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ')

    elif domain == 'www.chinatimes.com':

        # 中時新聞網
        item = soup.find_all('div', class_='article-body')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = [elem for elem in content]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ')

    elif domain == 'ctee.com.tw':

        # 工商時報
        item = soup.find_all('div', class_='entry-content clearfix single-post-content')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = [elem for elem in content]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ')

    elif domain == 'news.cnyes.com':

        # 鉅亨網
        item = soup.find_all('div', itemprop='articleBody')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = [elem for elem in content]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')

    elif domain == 'finance.ettoday.net':

        # ETtoday
        item = soup.find_all('div', itemprop='articleBody')[0].find_all('p')
        content = [elem.getText() for elem in item]
        content = [elem for elem in content]
        content = ''.join(content)
        content = content.replace('\r', ' ').replace('\n', ' ').replace(u'\xa0', ' ')

    else:

        # 未知domain
        content = ['unknow domain']

    return content

# 迴圈下載股票清單的Google新聞資料

In [11]:
stockNews = pd.DataFrame()

for iSearch in range(len(searchList)):

    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews = pd.concat([stockNews, df])

目前正在搜尋股票: 鴻海 在Google的新聞清單  進度: 1 / 51
目前正在下載: 鴻海 各家新聞  進度: 1 / 100
目前正在下載: 鴻海 各家新聞  進度: 2 / 100
目前正在下載: 鴻海 各家新聞  進度: 3 / 100
目前正在下載: 鴻海 各家新聞  進度: 4 / 100
目前正在下載: 鴻海 各家新聞  進度: 5 / 100
目前正在下載: 鴻海 各家新聞  進度: 6 / 100
目前正在下載: 鴻海 各家新聞  進度: 7 / 100
目前正在下載: 鴻海 各家新聞  進度: 8 / 100
目前正在下載: 鴻海 各家新聞  進度: 9 / 100
目前正在下載: 鴻海 各家新聞  進度: 10 / 100
目前正在下載: 鴻海 各家新聞  進度: 11 / 100
目前正在下載: 鴻海 各家新聞  進度: 12 / 100
目前正在下載: 鴻海 各家新聞  進度: 13 / 100
目前正在下載: 鴻海 各家新聞  進度: 14 / 100
目前正在下載: 鴻海 各家新聞  進度: 15 / 100
目前正在下載: 鴻海 各家新聞  進度: 16 / 100
目前正在下載: 鴻海 各家新聞  進度: 17 / 100
目前正在下載: 鴻海 各家新聞  進度: 18 / 100
目前正在下載: 鴻海 各家新聞  進度: 19 / 100
目前正在下載: 鴻海 各家新聞  進度: 20 / 100
目前正在下載: 鴻海 各家新聞  進度: 21 / 100
目前正在下載: 鴻海 各家新聞  進度: 22 / 100
目前正在下載: 鴻海 各家新聞  進度: 23 / 100
目前正在下載: 鴻海 各家新聞  進度: 24 / 100
目前正在下載: 鴻海 各家新聞  進度: 25 / 100
目前正在下載: 鴻海 各家新聞  進度: 26 / 100
目前正在下載: 鴻海 各家新聞  進度: 27 / 100
目前正在下載: 鴻海 各家新聞  進度: 28 / 100
目前正在下載: 鴻海 各家新聞  進度: 29 / 100
目前正在下載: 鴻海 各家新聞  進度: 30 / 100
目前正在下載: 鴻海 各家新聞  進度: 31 / 100
目前正在下載: 鴻海 各家新聞  進度: 32 / 100
目前正在下載: 鴻海 

目前正在下載: 台達電 各家新聞  進度: 66 / 100
目前正在下載: 台達電 各家新聞  進度: 67 / 100
目前正在下載: 台達電 各家新聞  進度: 68 / 100
目前正在下載: 台達電 各家新聞  進度: 69 / 100
目前正在下載: 台達電 各家新聞  進度: 70 / 100
目前正在下載: 台達電 各家新聞  進度: 71 / 100
目前正在下載: 台達電 各家新聞  進度: 72 / 100
目前正在下載: 台達電 各家新聞  進度: 73 / 100
目前正在下載: 台達電 各家新聞  進度: 74 / 100
目前正在下載: 台達電 各家新聞  進度: 75 / 100
目前正在下載: 台達電 各家新聞  進度: 76 / 100
目前正在下載: 台達電 各家新聞  進度: 77 / 100
目前正在下載: 台達電 各家新聞  進度: 78 / 100
目前正在下載: 台達電 各家新聞  進度: 79 / 100
目前正在下載: 台達電 各家新聞  進度: 80 / 100
目前正在下載: 台達電 各家新聞  進度: 81 / 100
目前正在下載: 台達電 各家新聞  進度: 82 / 100
目前正在下載: 台達電 各家新聞  進度: 83 / 100
目前正在下載: 台達電 各家新聞  進度: 84 / 100
目前正在下載: 台達電 各家新聞  進度: 85 / 100
目前正在下載: 台達電 各家新聞  進度: 86 / 100
目前正在下載: 台達電 各家新聞  進度: 87 / 100
目前正在下載: 台達電 各家新聞  進度: 88 / 100
目前正在下載: 台達電 各家新聞  進度: 89 / 100
目前正在下載: 台達電 各家新聞  進度: 90 / 100
目前正在下載: 台達電 各家新聞  進度: 91 / 100
目前正在下載: 台達電 各家新聞  進度: 92 / 100
目前正在下載: 台達電 各家新聞  進度: 93 / 100
目前正在下載: 台達電 各家新聞  進度: 94 / 100
目前正在下載: 台達電 各家新聞  進度: 95 / 100
目前正在下載: 台達電 各家新聞  進度: 96 / 100
目前正在下載: 台達電 各家新聞  進度: 97 / 100
目前正在下載: 

目前正在下載: 南亞 各家新聞  進度: 32 / 100
目前正在下載: 南亞 各家新聞  進度: 33 / 100
目前正在下載: 南亞 各家新聞  進度: 34 / 100
目前正在下載: 南亞 各家新聞  進度: 35 / 100
目前正在下載: 南亞 各家新聞  進度: 36 / 100
目前正在下載: 南亞 各家新聞  進度: 37 / 100
目前正在下載: 南亞 各家新聞  進度: 38 / 100
目前正在下載: 南亞 各家新聞  進度: 39 / 100
目前正在下載: 南亞 各家新聞  進度: 40 / 100
目前正在下載: 南亞 各家新聞  進度: 41 / 100
目前正在下載: 南亞 各家新聞  進度: 42 / 100
目前正在下載: 南亞 各家新聞  進度: 43 / 100
目前正在下載: 南亞 各家新聞  進度: 44 / 100
目前正在下載: 南亞 各家新聞  進度: 45 / 100
目前正在下載: 南亞 各家新聞  進度: 46 / 100
目前正在下載: 南亞 各家新聞  進度: 47 / 100
目前正在下載: 南亞 各家新聞  進度: 48 / 100
目前正在下載: 南亞 各家新聞  進度: 49 / 100
目前正在下載: 南亞 各家新聞  進度: 50 / 100
目前正在下載: 南亞 各家新聞  進度: 51 / 100
目前正在下載: 南亞 各家新聞  進度: 52 / 100
目前正在下載: 南亞 各家新聞  進度: 53 / 100
目前正在下載: 南亞 各家新聞  進度: 54 / 100
目前正在下載: 南亞 各家新聞  進度: 55 / 100
目前正在下載: 南亞 各家新聞  進度: 56 / 100
目前正在下載: 南亞 各家新聞  進度: 57 / 100
目前正在下載: 南亞 各家新聞  進度: 58 / 100
目前正在下載: 南亞 各家新聞  進度: 59 / 100
目前正在下載: 南亞 各家新聞  進度: 60 / 100
目前正在下載: 南亞 各家新聞  進度: 61 / 100
目前正在下載: 南亞 各家新聞  進度: 62 / 100
目前正在下載: 南亞 各家新聞  進度: 63 / 100
目前正在下載: 南亞 各家新聞  進度: 64 / 100
目前正在下載: 南亞

目前正在下載: 中信金 各家新聞  進度: 97 / 100
目前正在下載: 中信金 各家新聞  進度: 98 / 100
目前正在下載: 中信金 各家新聞  進度: 99 / 100
目前正在下載: 中信金 各家新聞  進度: 100 / 100
目前正在搜尋股票: 中鋼 在Google的新聞清單  進度: 9 / 51
目前正在下載: 中鋼 各家新聞  進度: 1 / 100
目前正在下載: 中鋼 各家新聞  進度: 2 / 100
目前正在下載: 中鋼 各家新聞  進度: 3 / 100
目前正在下載: 中鋼 各家新聞  進度: 4 / 100
目前正在下載: 中鋼 各家新聞  進度: 5 / 100
目前正在下載: 中鋼 各家新聞  進度: 6 / 100
目前正在下載: 中鋼 各家新聞  進度: 7 / 100
目前正在下載: 中鋼 各家新聞  進度: 8 / 100
目前正在下載: 中鋼 各家新聞  進度: 9 / 100
目前正在下載: 中鋼 各家新聞  進度: 10 / 100
目前正在下載: 中鋼 各家新聞  進度: 11 / 100
目前正在下載: 中鋼 各家新聞  進度: 12 / 100
目前正在下載: 中鋼 各家新聞  進度: 13 / 100
目前正在下載: 中鋼 各家新聞  進度: 14 / 100
目前正在下載: 中鋼 各家新聞  進度: 15 / 100
目前正在下載: 中鋼 各家新聞  進度: 16 / 100
目前正在下載: 中鋼 各家新聞  進度: 17 / 100
目前正在下載: 中鋼 各家新聞  進度: 18 / 100
目前正在下載: 中鋼 各家新聞  進度: 19 / 100
目前正在下載: 中鋼 各家新聞  進度: 20 / 100
目前正在下載: 中鋼 各家新聞  進度: 21 / 100
目前正在下載: 中鋼 各家新聞  進度: 22 / 100
目前正在下載: 中鋼 各家新聞  進度: 23 / 100
目前正在下載: 中鋼 各家新聞  進度: 24 / 100
目前正在下載: 中鋼 各家新聞  進度: 25 / 100
目前正在下載: 中鋼 各家新聞  進度: 26 / 100
目前正在下載: 中鋼 各家新聞  進度: 27 / 100
目前正在下載: 中鋼 各家新聞  進度: 28 / 100
目前正在下載

目前正在下載: 玉山金 各家新聞  進度: 62 / 100
目前正在下載: 玉山金 各家新聞  進度: 63 / 100
目前正在下載: 玉山金 各家新聞  進度: 64 / 100
目前正在下載: 玉山金 各家新聞  進度: 65 / 100
目前正在下載: 玉山金 各家新聞  進度: 66 / 100
目前正在下載: 玉山金 各家新聞  進度: 67 / 100
目前正在下載: 玉山金 各家新聞  進度: 68 / 100
目前正在下載: 玉山金 各家新聞  進度: 69 / 100
目前正在下載: 玉山金 各家新聞  進度: 70 / 100
目前正在下載: 玉山金 各家新聞  進度: 71 / 100
目前正在下載: 玉山金 各家新聞  進度: 72 / 100
目前正在下載: 玉山金 各家新聞  進度: 73 / 100
目前正在下載: 玉山金 各家新聞  進度: 74 / 100
目前正在下載: 玉山金 各家新聞  進度: 75 / 100
目前正在下載: 玉山金 各家新聞  進度: 76 / 100
目前正在下載: 玉山金 各家新聞  進度: 77 / 100
目前正在下載: 玉山金 各家新聞  進度: 78 / 100
目前正在下載: 玉山金 各家新聞  進度: 79 / 100
目前正在下載: 玉山金 各家新聞  進度: 80 / 100
目前正在下載: 玉山金 各家新聞  進度: 81 / 100
目前正在下載: 玉山金 各家新聞  進度: 82 / 100
目前正在下載: 玉山金 各家新聞  進度: 83 / 100
目前正在下載: 玉山金 各家新聞  進度: 84 / 100
目前正在下載: 玉山金 各家新聞  進度: 85 / 100
目前正在下載: 玉山金 各家新聞  進度: 86 / 100
目前正在下載: 玉山金 各家新聞  進度: 87 / 100
目前正在下載: 玉山金 各家新聞  進度: 88 / 100
目前正在下載: 玉山金 各家新聞  進度: 89 / 100
目前正在下載: 玉山金 各家新聞  進度: 90 / 100
目前正在下載: 玉山金 各家新聞  進度: 91 / 100
目前正在下載: 玉山金 各家新聞  進度: 92 / 100
目前正在下載: 玉山金 各家新聞  進度: 93 / 100
目前正在下載: 

...

目前正在下載: 元大金 各家新聞  進度: 51 / 100
目前正在下載: 元大金 各家新聞  進度: 52 / 100
目前正在下載: 元大金 各家新聞  進度: 53 / 100
目前正在下載: 元大金 各家新聞  進度: 54 / 100
目前正在下載: 元大金 各家新聞  進度: 55 / 100
目前正在下載: 元大金 各家新聞  進度: 56 / 100
目前正在下載: 元大金 各家新聞  進度: 57 / 100
目前正在下載: 元大金 各家新聞  進度: 58 / 100
目前正在下載: 元大金 各家新聞  進度: 59 / 100
目前正在下載: 元大金 各家新聞  進度: 60 / 100
目前正在下載: 元大金 各家新聞  進度: 61 / 100
目前正在下載: 元大金 各家新聞  進度: 62 / 100
目前正在下載: 元大金 各家新聞  進度: 63 / 100
目前正在下載: 元大金 各家新聞  進度: 64 / 100
目前正在下載: 元大金 各家新聞  進度: 65 / 100
目前正在下載: 元大金 各家新聞  進度: 66 / 100
目前正在下載: 元大金 各家新聞  進度: 67 / 100
目前正在下載: 元大金 各家新聞  進度: 68 / 100
目前正在下載: 元大金 各家新聞  進度: 69 / 100
目前正在下載: 元大金 各家新聞  進度: 70 / 100
目前正在下載: 元大金 各家新聞  進度: 71 / 100
目前正在下載: 元大金 各家新聞  進度: 72 / 100
目前正在下載: 元大金 各家新聞  進度: 73 / 100
目前正在下載: 元大金 各家新聞  進度: 74 / 100
目前正在下載: 元大金 各家新聞  進度: 75 / 100
目前正在下載: 元大金 各家新聞  進度: 76 / 100
目前正在下載: 元大金 各家新聞  進度: 77 / 100
目前正在下載: 元大金 各家新聞  進度: 78 / 100
目前正在下載: 元大金 各家新聞  進度: 79 / 100
目前正在下載: 元大金 各家新聞  進度: 80 / 100
目前正在下載: 元大金 各家新聞  進度: 81 / 100
目前正在下載: 元大金 各家新聞  進度: 82 / 100
目前正在下載: 

目前正在下載: 台泥 各家新聞  進度: 6 / 100
目前正在下載: 台泥 各家新聞  進度: 7 / 100
目前正在下載: 台泥 各家新聞  進度: 8 / 100
目前正在下載: 台泥 各家新聞  進度: 9 / 100
目前正在下載: 台泥 各家新聞  進度: 10 / 100
目前正在下載: 台泥 各家新聞  進度: 11 / 100
目前正在下載: 台泥 各家新聞  進度: 12 / 100
目前正在下載: 台泥 各家新聞  進度: 13 / 100
目前正在下載: 台泥 各家新聞  進度: 14 / 100
目前正在下載: 台泥 各家新聞  進度: 15 / 100
目前正在下載: 台泥 各家新聞  進度: 16 / 100
目前正在下載: 台泥 各家新聞  進度: 17 / 100
目前正在下載: 台泥 各家新聞  進度: 18 / 100
目前正在下載: 台泥 各家新聞  進度: 19 / 100
目前正在下載: 台泥 各家新聞  進度: 20 / 100
目前正在下載: 台泥 各家新聞  進度: 21 / 100
目前正在下載: 台泥 各家新聞  進度: 22 / 100
目前正在下載: 台泥 各家新聞  進度: 23 / 100
目前正在下載: 台泥 各家新聞  進度: 24 / 100
目前正在下載: 台泥 各家新聞  進度: 25 / 100
目前正在下載: 台泥 各家新聞  進度: 26 / 100
目前正在下載: 台泥 各家新聞  進度: 27 / 100
目前正在下載: 台泥 各家新聞  進度: 28 / 100
目前正在下載: 台泥 各家新聞  進度: 29 / 100
目前正在下載: 台泥 各家新聞  進度: 30 / 100
目前正在下載: 台泥 各家新聞  進度: 31 / 100
目前正在下載: 台泥 各家新聞  進度: 32 / 100
目前正在下載: 台泥 各家新聞  進度: 33 / 100
目前正在下載: 台泥 各家新聞  進度: 34 / 100
目前正在下載: 台泥 各家新聞  進度: 35 / 100
目前正在下載: 台泥 各家新聞  進度: 36 / 100
目前正在下載: 台泥 各家新聞  進度: 37 / 100
目前正在下載: 台泥 各家新聞  進度: 38 / 100
目前正在下載: 台泥 各家新

目前正在下載: 開發金 各家新聞  進度: 75 / 100
目前正在下載: 開發金 各家新聞  進度: 76 / 100
目前正在下載: 開發金 各家新聞  進度: 77 / 100
目前正在下載: 開發金 各家新聞  進度: 78 / 100
目前正在下載: 開發金 各家新聞  進度: 79 / 100
目前正在下載: 開發金 各家新聞  進度: 80 / 100
目前正在下載: 開發金 各家新聞  進度: 81 / 100
目前正在下載: 開發金 各家新聞  進度: 82 / 100
目前正在下載: 開發金 各家新聞  進度: 83 / 100
目前正在下載: 開發金 各家新聞  進度: 84 / 100
目前正在下載: 開發金 各家新聞  進度: 85 / 100
目前正在下載: 開發金 各家新聞  進度: 86 / 100
目前正在下載: 開發金 各家新聞  進度: 87 / 100
目前正在下載: 開發金 各家新聞  進度: 88 / 100
目前正在下載: 開發金 各家新聞  進度: 89 / 100
目前正在下載: 開發金 各家新聞  進度: 90 / 100
目前正在下載: 開發金 各家新聞  進度: 91 / 100
目前正在下載: 開發金 各家新聞  進度: 92 / 100
目前正在下載: 開發金 各家新聞  進度: 93 / 100
目前正在下載: 開發金 各家新聞  進度: 94 / 100
目前正在下載: 開發金 各家新聞  進度: 95 / 100
目前正在下載: 開發金 各家新聞  進度: 96 / 100
目前正在下載: 開發金 各家新聞  進度: 97 / 100
目前正在下載: 開發金 各家新聞  進度: 98 / 100
目前正在下載: 開發金 各家新聞  進度: 99 / 100
目前正在下載: 開發金 各家新聞  進度: 100 / 100
目前正在搜尋股票: 和泰車 在Google的新聞清單  進度: 25 / 51
目前正在下載: 和泰車 各家新聞  進度: 1 / 100
目前正在下載: 和泰車 各家新聞  進度: 2 / 100
目前正在下載: 和泰車 各家新聞  進度: 3 / 100
目前正在下載: 和泰車 各家新聞  進度: 4 / 100
目前正在下載: 和泰車 各家新聞  進度: 5 / 100
目前正

目前正在下載: 永豐金 各家新聞  進度: 37 / 100
目前正在下載: 永豐金 各家新聞  進度: 38 / 100
目前正在下載: 永豐金 各家新聞  進度: 39 / 100
目前正在下載: 永豐金 各家新聞  進度: 40 / 100
目前正在下載: 永豐金 各家新聞  進度: 41 / 100
目前正在下載: 永豐金 各家新聞  進度: 42 / 100
目前正在下載: 永豐金 各家新聞  進度: 43 / 100
目前正在下載: 永豐金 各家新聞  進度: 44 / 100
目前正在下載: 永豐金 各家新聞  進度: 45 / 100
目前正在下載: 永豐金 各家新聞  進度: 46 / 100
目前正在下載: 永豐金 各家新聞  進度: 47 / 100
目前正在下載: 永豐金 各家新聞  進度: 48 / 100
目前正在下載: 永豐金 各家新聞  進度: 49 / 100
目前正在下載: 永豐金 各家新聞  進度: 50 / 100
目前正在下載: 永豐金 各家新聞  進度: 51 / 100
目前正在下載: 永豐金 各家新聞  進度: 52 / 100
目前正在下載: 永豐金 各家新聞  進度: 53 / 100
目前正在下載: 永豐金 各家新聞  進度: 54 / 100
目前正在下載: 永豐金 各家新聞  進度: 55 / 100
目前正在下載: 永豐金 各家新聞  進度: 56 / 100
目前正在下載: 永豐金 各家新聞  進度: 57 / 100
目前正在下載: 永豐金 各家新聞  進度: 58 / 100
目前正在下載: 永豐金 各家新聞  進度: 59 / 100
目前正在下載: 永豐金 各家新聞  進度: 60 / 100
目前正在下載: 永豐金 各家新聞  進度: 61 / 100
目前正在下載: 永豐金 各家新聞  進度: 62 / 100
目前正在下載: 永豐金 各家新聞  進度: 63 / 100
目前正在下載: 永豐金 各家新聞  進度: 64 / 100
目前正在下載: 永豐金 各家新聞  進度: 65 / 100
目前正在下載: 永豐金 各家新聞  進度: 66 / 100
目前正在下載: 永豐金 各家新聞  進度: 67 / 100
目前正在下載: 永豐金 各家新聞  進度: 68 / 100
目前正在下載: 

IndexError: list index out of range

In [17]:
# 欲下載新聞的股票關鍵字清單
company_str = '台灣大	, 統一超	, 上海商銀	, 台新金	, 欣興	, 聯詠	, 長榮	, 國巨	, 臺股期貨	, 友達	, 矽力*-KY	, 研華	, 瑞昱	, 亞德客-KY	, 彰銀	, 遠傳	, 陽明	, 台塑化	, 萬海	, 豐泰	, 南亞科	, 南電	,'
searchList = company_str.strip('\t,').split('\t, ')
searchList.append('台灣50')
# 新聞下載起始日
StartDate = '2010-01-01'

In [19]:
stockNews2 = pd.DataFrame()

for iSearch in range(len(searchList)):

    print('目前正在搜尋股票: ' + searchList[iSearch] +
          ' 在Google的新聞清單  進度: ' + str(iSearch + 1) + ' / ' + str(len(searchList)))

    # 建立搜尋網址
    url = 'https://news.google.com/news/rss/search/section/q/' + \
          searchList[iSearch] + '/?hl=zh-tw&gl=TW&ned=zh-tw_tw'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'xml')
    item = soup.find_all('item')
    rows = [arrangeGoogleNews(elem) for elem in item]

    # 組成pandas
    df = pd.DataFrame(data=rows, columns=['title', 'link', 'pub_date', 'description', 'source'])
    # 新增時間戳記欄位
    df.insert(0, 'search_time', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), True)
    # 新增搜尋字串
    df.insert(1, 'search_key', searchList[iSearch], True)
    # 篩選最近的新聞
    df['pub_date'] = df['pub_date'].astype('datetime64[ns]')
    df = df[df['pub_date'] >= StartDate].reset_index(drop=True)

    # 迴圈爬取新聞內容
    content = list()
    for iLink in range(len(df['link'])):

        print('目前正在下載: ' + searchList[iSearch] +
              ' 各家新聞  進度: ' + str(iLink + 1) + ' / ' + str(len(df['link'])))

        content.append(beautifulSoupNews(url=df['link'][iLink]))

    # 新增新聞內容欄位
    df['content'] = content

    # 儲存資料
    stockNews2 = pd.concat([stockNews2, df])

目前正在搜尋股票: 台灣大 在Google的新聞清單  進度: 1 / 23
目前正在下載: 台灣大 各家新聞  進度: 1 / 100
目前正在下載: 台灣大 各家新聞  進度: 2 / 100
目前正在下載: 台灣大 各家新聞  進度: 3 / 100
目前正在下載: 台灣大 各家新聞  進度: 4 / 100
目前正在下載: 台灣大 各家新聞  進度: 5 / 100
目前正在下載: 台灣大 各家新聞  進度: 6 / 100
目前正在下載: 台灣大 各家新聞  進度: 7 / 100
目前正在下載: 台灣大 各家新聞  進度: 8 / 100
目前正在下載: 台灣大 各家新聞  進度: 9 / 100
目前正在下載: 台灣大 各家新聞  進度: 10 / 100
目前正在下載: 台灣大 各家新聞  進度: 11 / 100
目前正在下載: 台灣大 各家新聞  進度: 12 / 100
目前正在下載: 台灣大 各家新聞  進度: 13 / 100
目前正在下載: 台灣大 各家新聞  進度: 14 / 100
目前正在下載: 台灣大 各家新聞  進度: 15 / 100
目前正在下載: 台灣大 各家新聞  進度: 16 / 100
目前正在下載: 台灣大 各家新聞  進度: 17 / 100
目前正在下載: 台灣大 各家新聞  進度: 18 / 100
目前正在下載: 台灣大 各家新聞  進度: 19 / 100
目前正在下載: 台灣大 各家新聞  進度: 20 / 100
目前正在下載: 台灣大 各家新聞  進度: 21 / 100
目前正在下載: 台灣大 各家新聞  進度: 22 / 100
目前正在下載: 台灣大 各家新聞  進度: 23 / 100
目前正在下載: 台灣大 各家新聞  進度: 24 / 100
目前正在下載: 台灣大 各家新聞  進度: 25 / 100
目前正在下載: 台灣大 各家新聞  進度: 26 / 100
目前正在下載: 台灣大 各家新聞  進度: 27 / 100
目前正在下載: 台灣大 各家新聞  進度: 28 / 100
目前正在下載: 台灣大 各家新聞  進度: 29 / 100
目前正在下載: 台灣大 各家新聞  進度: 30 / 100
目前正在下載: 台灣大 各家新聞  進度: 31 / 100
目前正在下載: 台

目前正在下載: 上海商銀 各家新聞  進度: 61 / 100
目前正在下載: 上海商銀 各家新聞  進度: 62 / 100
目前正在下載: 上海商銀 各家新聞  進度: 63 / 100
目前正在下載: 上海商銀 各家新聞  進度: 64 / 100
目前正在下載: 上海商銀 各家新聞  進度: 65 / 100
目前正在下載: 上海商銀 各家新聞  進度: 66 / 100
目前正在下載: 上海商銀 各家新聞  進度: 67 / 100
目前正在下載: 上海商銀 各家新聞  進度: 68 / 100
目前正在下載: 上海商銀 各家新聞  進度: 69 / 100
目前正在下載: 上海商銀 各家新聞  進度: 70 / 100
目前正在下載: 上海商銀 各家新聞  進度: 71 / 100
目前正在下載: 上海商銀 各家新聞  進度: 72 / 100
目前正在下載: 上海商銀 各家新聞  進度: 73 / 100
目前正在下載: 上海商銀 各家新聞  進度: 74 / 100
目前正在下載: 上海商銀 各家新聞  進度: 75 / 100
目前正在下載: 上海商銀 各家新聞  進度: 76 / 100
目前正在下載: 上海商銀 各家新聞  進度: 77 / 100
目前正在下載: 上海商銀 各家新聞  進度: 78 / 100
目前正在下載: 上海商銀 各家新聞  進度: 79 / 100
目前正在下載: 上海商銀 各家新聞  進度: 80 / 100
目前正在下載: 上海商銀 各家新聞  進度: 81 / 100
目前正在下載: 上海商銀 各家新聞  進度: 82 / 100
目前正在下載: 上海商銀 各家新聞  進度: 83 / 100
目前正在下載: 上海商銀 各家新聞  進度: 84 / 100
目前正在下載: 上海商銀 各家新聞  進度: 85 / 100
目前正在下載: 上海商銀 各家新聞  進度: 86 / 100
目前正在下載: 上海商銀 各家新聞  進度: 87 / 100
目前正在下載: 上海商銀 各家新聞  進度: 88 / 100
目前正在下載: 上海商銀 各家新聞  進度: 89 / 100
目前正在下載: 上海商銀 各家新聞  進度: 90 / 100
目前正在下載: 上海商銀 各家新聞  進度: 91 / 100
目前正在下載: 

目前正在下載: 聯詠 各家新聞  進度: 26 / 100
目前正在下載: 聯詠 各家新聞  進度: 27 / 100
目前正在下載: 聯詠 各家新聞  進度: 28 / 100
目前正在下載: 聯詠 各家新聞  進度: 29 / 100
目前正在下載: 聯詠 各家新聞  進度: 30 / 100
目前正在下載: 聯詠 各家新聞  進度: 31 / 100
目前正在下載: 聯詠 各家新聞  進度: 32 / 100
目前正在下載: 聯詠 各家新聞  進度: 33 / 100
目前正在下載: 聯詠 各家新聞  進度: 34 / 100
目前正在下載: 聯詠 各家新聞  進度: 35 / 100
目前正在下載: 聯詠 各家新聞  進度: 36 / 100
目前正在下載: 聯詠 各家新聞  進度: 37 / 100
目前正在下載: 聯詠 各家新聞  進度: 38 / 100
目前正在下載: 聯詠 各家新聞  進度: 39 / 100
目前正在下載: 聯詠 各家新聞  進度: 40 / 100
目前正在下載: 聯詠 各家新聞  進度: 41 / 100
目前正在下載: 聯詠 各家新聞  進度: 42 / 100
目前正在下載: 聯詠 各家新聞  進度: 43 / 100
目前正在下載: 聯詠 各家新聞  進度: 44 / 100
目前正在下載: 聯詠 各家新聞  進度: 45 / 100
目前正在下載: 聯詠 各家新聞  進度: 46 / 100
目前正在下載: 聯詠 各家新聞  進度: 47 / 100
目前正在下載: 聯詠 各家新聞  進度: 48 / 100
目前正在下載: 聯詠 各家新聞  進度: 49 / 100
目前正在下載: 聯詠 各家新聞  進度: 50 / 100
目前正在下載: 聯詠 各家新聞  進度: 51 / 100
目前正在下載: 聯詠 各家新聞  進度: 52 / 100
目前正在下載: 聯詠 各家新聞  進度: 53 / 100
目前正在下載: 聯詠 各家新聞  進度: 54 / 100
目前正在下載: 聯詠 各家新聞  進度: 55 / 100
目前正在下載: 聯詠 各家新聞  進度: 56 / 100
目前正在下載: 聯詠 各家新聞  進度: 57 / 100
目前正在下載: 聯詠 各家新聞  進度: 58 / 100
目前正在下載: 聯詠

目前正在下載: 國巨 各家新聞  進度: 98 / 100
目前正在下載: 國巨 各家新聞  進度: 99 / 100
目前正在下載: 國巨 各家新聞  進度: 100 / 100
目前正在搜尋股票: 臺股期貨 在Google的新聞清單  進度: 9 / 23
目前正在下載: 臺股期貨 各家新聞  進度: 1 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 2 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 3 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 4 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 5 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 6 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 7 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 8 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 9 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 10 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 11 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 12 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 13 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 14 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 15 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 16 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 17 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 18 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 19 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 20 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 21 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 22 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 23 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 24 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 25 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 26 / 100
目前正在下載: 臺股期貨 各家新聞  進度: 27 / 100
目前正在下載: 臺股期貨 各

目前正在下載: 矽力*-KY 各家新聞  進度: 55 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 56 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 57 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 58 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 59 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 60 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 61 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 62 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 63 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 64 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 65 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 66 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 67 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 68 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 69 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 70 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 71 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 72 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 73 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 74 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 75 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 76 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 77 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 78 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 79 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 80 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 81 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 82 / 100
目前正在下載: 矽力*-KY 各家新聞  進度: 83 / 100
目前正在下載: 矽力*-KY

目前正在下載: 亞德客-KY 各家新聞  進度: 18 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 19 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 20 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 21 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 22 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 23 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 24 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 25 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 26 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 27 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 28 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 29 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 30 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 31 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 32 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 33 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 34 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 35 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 36 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 37 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 38 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 39 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 40 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 41 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 42 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 43 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 44 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 45 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 46 / 97
目前正在下載: 亞德客-KY 各家新聞  進度: 47 / 97
目前正在下載: 亞德

目前正在下載: 遠傳 各家新聞  進度: 85 / 100
目前正在下載: 遠傳 各家新聞  進度: 86 / 100
目前正在下載: 遠傳 各家新聞  進度: 87 / 100
目前正在下載: 遠傳 各家新聞  進度: 88 / 100
目前正在下載: 遠傳 各家新聞  進度: 89 / 100
目前正在下載: 遠傳 各家新聞  進度: 90 / 100
目前正在下載: 遠傳 各家新聞  進度: 91 / 100
目前正在下載: 遠傳 各家新聞  進度: 92 / 100
目前正在下載: 遠傳 各家新聞  進度: 93 / 100
目前正在下載: 遠傳 各家新聞  進度: 94 / 100
目前正在下載: 遠傳 各家新聞  進度: 95 / 100
目前正在下載: 遠傳 各家新聞  進度: 96 / 100
目前正在下載: 遠傳 各家新聞  進度: 97 / 100
目前正在下載: 遠傳 各家新聞  進度: 98 / 100
目前正在下載: 遠傳 各家新聞  進度: 99 / 100
目前正在下載: 遠傳 各家新聞  進度: 100 / 100
目前正在搜尋股票: 陽明 在Google的新聞清單  進度: 17 / 23
目前正在下載: 陽明 各家新聞  進度: 1 / 100
目前正在下載: 陽明 各家新聞  進度: 2 / 100
目前正在下載: 陽明 各家新聞  進度: 3 / 100
目前正在下載: 陽明 各家新聞  進度: 4 / 100
目前正在下載: 陽明 各家新聞  進度: 5 / 100
目前正在下載: 陽明 各家新聞  進度: 6 / 100
目前正在下載: 陽明 各家新聞  進度: 7 / 100
目前正在下載: 陽明 各家新聞  進度: 8 / 100
目前正在下載: 陽明 各家新聞  進度: 9 / 100
目前正在下載: 陽明 各家新聞  進度: 10 / 100
目前正在下載: 陽明 各家新聞  進度: 11 / 100
目前正在下載: 陽明 各家新聞  進度: 12 / 100
目前正在下載: 陽明 各家新聞  進度: 13 / 100
目前正在下載: 陽明 各家新聞  進度: 14 / 100
目前正在下載: 陽明 各家新聞  進度: 15 / 100
目前正在下載: 陽明 各家新聞  進度: 16 / 100
目前正在下載: 陽

目前正在下載: 萬海 各家新聞  進度: 52 / 100
目前正在下載: 萬海 各家新聞  進度: 53 / 100
目前正在下載: 萬海 各家新聞  進度: 54 / 100
目前正在下載: 萬海 各家新聞  進度: 55 / 100
目前正在下載: 萬海 各家新聞  進度: 56 / 100
目前正在下載: 萬海 各家新聞  進度: 57 / 100
目前正在下載: 萬海 各家新聞  進度: 58 / 100
目前正在下載: 萬海 各家新聞  進度: 59 / 100
目前正在下載: 萬海 各家新聞  進度: 60 / 100
目前正在下載: 萬海 各家新聞  進度: 61 / 100
目前正在下載: 萬海 各家新聞  進度: 62 / 100
目前正在下載: 萬海 各家新聞  進度: 63 / 100
目前正在下載: 萬海 各家新聞  進度: 64 / 100
目前正在下載: 萬海 各家新聞  進度: 65 / 100
目前正在下載: 萬海 各家新聞  進度: 66 / 100
目前正在下載: 萬海 各家新聞  進度: 67 / 100
目前正在下載: 萬海 各家新聞  進度: 68 / 100
目前正在下載: 萬海 各家新聞  進度: 69 / 100
目前正在下載: 萬海 各家新聞  進度: 70 / 100
目前正在下載: 萬海 各家新聞  進度: 71 / 100
目前正在下載: 萬海 各家新聞  進度: 72 / 100
目前正在下載: 萬海 各家新聞  進度: 73 / 100
目前正在下載: 萬海 各家新聞  進度: 74 / 100
目前正在下載: 萬海 各家新聞  進度: 75 / 100
目前正在下載: 萬海 各家新聞  進度: 76 / 100
目前正在下載: 萬海 各家新聞  進度: 77 / 100
目前正在下載: 萬海 各家新聞  進度: 78 / 100
目前正在下載: 萬海 各家新聞  進度: 79 / 100
目前正在下載: 萬海 各家新聞  進度: 80 / 100
目前正在下載: 萬海 各家新聞  進度: 81 / 100
目前正在下載: 萬海 各家新聞  進度: 82 / 100
目前正在下載: 萬海 各家新聞  進度: 83 / 100
目前正在下載: 萬海 各家新聞  進度: 84 / 100
目前正在下載: 萬海

目前正在下載: 南電 各家新聞  進度: 19 / 100
目前正在下載: 南電 各家新聞  進度: 20 / 100
目前正在下載: 南電 各家新聞  進度: 21 / 100
目前正在下載: 南電 各家新聞  進度: 22 / 100
目前正在下載: 南電 各家新聞  進度: 23 / 100
目前正在下載: 南電 各家新聞  進度: 24 / 100
目前正在下載: 南電 各家新聞  進度: 25 / 100
目前正在下載: 南電 各家新聞  進度: 26 / 100
目前正在下載: 南電 各家新聞  進度: 27 / 100
目前正在下載: 南電 各家新聞  進度: 28 / 100
目前正在下載: 南電 各家新聞  進度: 29 / 100
目前正在下載: 南電 各家新聞  進度: 30 / 100
目前正在下載: 南電 各家新聞  進度: 31 / 100
目前正在下載: 南電 各家新聞  進度: 32 / 100
目前正在下載: 南電 各家新聞  進度: 33 / 100
目前正在下載: 南電 各家新聞  進度: 34 / 100
目前正在下載: 南電 各家新聞  進度: 35 / 100
目前正在下載: 南電 各家新聞  進度: 36 / 100
目前正在下載: 南電 各家新聞  進度: 37 / 100
目前正在下載: 南電 各家新聞  進度: 38 / 100
目前正在下載: 南電 各家新聞  進度: 39 / 100
目前正在下載: 南電 各家新聞  進度: 40 / 100
目前正在下載: 南電 各家新聞  進度: 41 / 100
目前正在下載: 南電 各家新聞  進度: 42 / 100
目前正在下載: 南電 各家新聞  進度: 43 / 100
目前正在下載: 南電 各家新聞  進度: 44 / 100
目前正在下載: 南電 各家新聞  進度: 45 / 100
目前正在下載: 南電 各家新聞  進度: 46 / 100
目前正在下載: 南電 各家新聞  進度: 47 / 100
目前正在下載: 南電 各家新聞  進度: 48 / 100
目前正在下載: 南電 各家新聞  進度: 49 / 100
目前正在下載: 南電 各家新聞  進度: 50 / 100
目前正在下載: 南電 各家新聞  進度: 51 / 100
目前正在下載: 南電

In [22]:
tmp = stockNews.copy()
tmp2 = stockNews2.copy()

In [25]:
StockNews = pd.concat([tmp, tmp2])
StockNews

,search_time,search_key,title,link,pub_date,description,source,content
0,2022-10-27 14:04:23,鴻海,100塊的鴻海買嗎？ 專家：若這2產業佈局成功 股價上看170元 - 聯合新聞網,https://news.google.com/__i/rss/rd/articles/CB...,2022-10-26 05:29:14,"<a href=""https://news.google.com/__i/rss/rd/ar...",聯合新聞網,財經專家黃世聰在節目「聰明理財大小世」中分享，若鴻海（2317）所額外布局的半導體和電...
1,2022-10-27 14:04:23,鴻海,鴻海攜手揚斯敦州大 成立電動車培訓暨創新中心 - Anue鉅亨,https://news.google.com/__i/rss/rd/articles/CB...,2022-10-27 03:41:55,"<a href=""https://news.google.com/__i/rss/rd/ar...",Anue鉅亨,揚斯敦州立大學 (YSU) 和鴻海 (2317-TW) 今 (27) 日共同宣布，雙方建立合...
2,2022-10-27 14:04:23,鴻海,鴻海衝電動車 供應鏈樂 - 聯合新聞網,https://news.google.com/__i/rss/rd/articles/CB...,2022-10-26 15:48:43,"<a href=""https://news.google.com/__i/rss/rd/ar...",聯合新聞網,納智捷n7為採用鴻海Model C平台模組推出的第一台量產電動車款，此款電動車委由裕隆（22...
3,2022-10-27 14:04:23,鴻海,【鴻海電車拚兆元1】急行軍3年造5車 汽車業耆老一句話點破鴻海盲點 - Yahoo奇摩新聞,https://news.google.com/__i/rss/rd/articles/CB...,2022-10-24 21:58:58,"<a href=""https://news.google.com/__i/rss/rd/ar...",Yahoo奇摩新聞,靠著鴻海速度僅花3年造出5台電動車，讓董事長劉揚偉意氣風發高喊2025年電動車相關營收要衝上...
4,2022-10-27 14:04:23,鴻海,近距離看鴻海電動車！Model B與V今起在三創生活園區展示 - 聯合新聞網,https://news.google.com/__i/rss/rd/articles/CB...,2022-10-20 06:15:13,"<a href=""https://news.google.com/__i/rss/rd/ar...",聯合新聞網,鴻海（2317）官網今（20）日表示，由納智捷（LUXGEN）邀請鴻海展示最新款電動車平...
...,...,...,...,...,...,...,...,...
95,2022-10-27 18:37:22,台灣50,0050在紅什麼？空頭也能賺？投資達人教「這兩招」：多頭能賺4成、空頭獲利也有15% - 今...,https://news.google.com/__i/rss/rd/articles/CB...,2022-03-11 08:00:00,"<a href=""https://news.google.com/__i/rss/rd/ar...",今周刊-在今天看見明天,[unknow domain]
96,2022-10-27 18:37:22,台灣50,看好台股下半年上漲機率達9成 00912重新定義台灣50 - Anue鉅亨,https://news.google.com/__i/rss/rd/articles/CB...,2022-06-06 07:00:00,"<a href=""https://news.google.com/__i/rss/rd/ar...",Anue鉅亨,台股今 (2022) 年以來波動劇烈，但中信投信認為，台股已歷經大幅度修正、明顯超跌，預期未...
97,2022-10-27 18:37:22,台灣50,台灣50(0050)這樣買就穩賺？副教授實測網傳2手法：景氣燈號、KD值操作，結果報酬率分別...,https://news.google.com/__i/rss/rd/articles/CB...,2022-03-31 07:00:00,"<a href=""https://news.google.com/__i/rss/rd/ar...",今周刊-在今天看見明天,[unknow domain]
98,2022-10-27 18:37:22,台灣50,"最強操作 三竹靠台灣50反一大賺1,715萬 - 聯合新聞網",https://news.google.com/__i/rss/rd/articles/CB...,2022-07-06 07:00:00,"<a href=""https://news.google.com/__i/rss/rd/ar...",聯合新聞網,三竹（8284）今（6）日在公開資訊觀測站公告處分元大台灣50反1（00632R），合計賣出...


In [26]:
StockNews.to_pickle("NewsData.pkl")